# TSA Chapter 4: ACF/PACF After Differencing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_acf_pacf/TSA_ch4_acf_pacf.ipynb)

ACF and PACF of doubly differenced log airline passengers for SARIMA model identification.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
# Doubly differenced log series
diff_both = log_passengers.diff().diff(12).dropna()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# ACF
acf_vals = acf(diff_both, nlags=36)
ci = 1.96 / np.sqrt(len(diff_both))
bar_colors_acf = [RED if i > 0 and i % 12 == 0 else BLUE for i in range(len(acf_vals))]
axes[0].bar(range(len(acf_vals)), acf_vals, color=bar_colors_acf, width=0.4)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].axhline(y=ci, color=GRAY, linestyle='--', alpha=0.7, label='95% CI')
axes[0].axhline(y=-ci, color=GRAY, linestyle='--', alpha=0.7)
for lag in [12, 24, 36]:
    if lag < len(acf_vals):
        axes[0].axvline(x=lag, color=ORANGE, linestyle=':', alpha=0.4)
axes[0].set_title('ACF of $\\Delta\\Delta_{12}\\log(Y_t)$', fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False)

# PACF
pacf_vals = pacf(diff_both, nlags=36)
bar_colors_pacf = [RED if i > 0 and i % 12 == 0 else GREEN for i in range(len(pacf_vals))]
axes[1].bar(range(len(pacf_vals)), pacf_vals, color=bar_colors_pacf, width=0.4)
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].axhline(y=ci, color=GRAY, linestyle='--', alpha=0.7, label='95% CI')
axes[1].axhline(y=-ci, color=GRAY, linestyle='--', alpha=0.7)
for lag in [12, 24, 36]:
    if lag < len(pacf_vals):
        axes[1].axvline(x=lag, color=ORANGE, linestyle=':', alpha=0.4)
axes[1].set_title('PACF of $\\Delta\\Delta_{12}\\log(Y_t)$', fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('PACF')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), frameon=False)

plt.tight_layout(); save_chart(fig, 'ch4_acf_pacf'); plt.show()